In [1]:
import gumpy

import numpy, time, copy

%load_ext autoreload
%autoreload 2

In [ ]:

import scipy
scipy.__version__
scipy.__path__

In [18]:
import Bio
Bio.__path__

['/Library/Python/3.8/site-packages/biopython-1.78-py3.8-macosx-10.14.6-x86_64.egg/Bio']

In [13]:
numpy.__version__
# numpy.__path__

'1.20.2'

# Let's be topical and use the SARS-CoV-2 Wuhan reference genome

In [4]:
gumpy.__path__

['/Users/fowler/packages/gumpy/gumpy']

In [ ]:
tb=gumpy.Genome('config/NC_000962.3.gbk')

 61%|██████    | 48/79 [5:36:42<5:49:23, 676.23s/it]

In [2]:
covid=gumpy.Genome('config/NC_045512.2.gbk',verbose=True, default_promoter_length=10)

100%|██████████| 12/12 [00:00<00:00, 256.91it/s]

       parse genbank  0.049 s
       define arrays  0.034 s
            promoter  0.004 s
        create genes  0.049 s


The `covid` Genome object has pretty print since `__repr__` is overloaded

In [3]:
covid.at_index(21765)

['S']

In [4]:
[covid.nucleotide_index==21765]

[array([False, False, False, ..., False, False, False])]

In [9]:
covid.genes['S'].nucleotide_number[covid.genes['S'].index==21765]

array([203])

In [325]:
print(covid)

NC_045512
NC_045512.2
Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
29903 bases
attaaa...aaaaaa
all genes/loci have been included



Much of this information it has parsed from the annotations the GenBank file

In [326]:
covid.annotations

{'molecule_type': 'ss-RNA',
 'topology': 'linear',
 'data_file_division': 'VRL',
 'date': '18-JUL-2020',
 'accessions': ['NC_045512'],
 'sequence_version': 2,
 'keywords': ['RefSeq'],
 'source': 'Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)',
 'organism': 'Severe acute respiratory syndrome coronavirus 2',
 'taxonomy': ['Viruses',
  'Riboviria',
  'Orthornavirae',
  'Pisuviricota',
  'Pisoniviricetes',
  'Nidovirales',
  'Cornidovirineae',
  'Coronaviridae',
  'Orthocoronavirinae',
  'Betacoronavirus',
  'Sarbecovirus'],
 'references': [Reference(title='A new coronavirus associated with human respiratory disease in China', ...),
  Reference(title='Programmed ribosomal frameshifting in decoding the SARS-CoV genome', ...),
  Reference(title='The structure of a rigorously conserved RNA element within the SARS virus genome', ...),
  Reference(title="A phylogenetically conserved hairpin-type 3' untranslated region pseudoknot functions in coronavirus RNA replication", ...),
  

In [327]:
print(covid.name)

NC_045512


In [328]:
print(covid.id)

NC_045512.2


In [329]:
print(covid.description)

Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome


In [330]:
print(covid.length)

29903


Since a genbank file only ever stores a single sequence, regardless of whether it is dsDNA or ssRNA or whatever, this is stored, along with the indices, in `numpy` arrays

In [331]:
covid.nucleotide_sequence

array(['a', 't', 't', ..., 'a', 'a', 'a'], dtype='<U1')

In [333]:
covid.nucleotide_index

array([    1,     2,     3, ..., 29901, 29902, 29903])

This permits fancy-indexing in a Pythonic/numpy manner

In [336]:
covid.nucleotide_sequence[covid.nucleotide_index==1000]

array(['t'], dtype='<U1')

The `Genome` class has some useful functions

In [337]:
covid.contains_gene('S')

True

This method tells us which genes/loci are involved at this position 

In [338]:
covid.at_index(1000)

['ORF1ab', 'ORF1ab_2']

Then we can also save the genome as a `FASTA` file, which comes with some options (which have sensible defaults)

In [339]:
covid.save_fasta('covid.fasta',\
                 compression=False,\
                 chars_per_line=80,\
                 nucleotides_uppercase=False)

Finally we could save the sequence as a compressed `npz` file which can be loaded into memory **much** faster than a FASTA file. 

In [340]:
covid.save_sequence('covid.npz')

In [341]:
a=numpy.load('covid.npz')
a['sequence']

array(['a', 't', 't', ..., 'a', 'a', 'a'], dtype='<U1')

In [342]:
%timeit numpy.load('covid.npz')

109 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


As a comparison, let's try loading the FASTA file with BioPython - on my machine the above is about 5x faster

In [343]:
from Bio import SeqIO

In [344]:
start=time.time()
for seq_record in SeqIO.parse("covid.fasta", "fasta"):
    covid_sequence=seq_record.seq
print("%.1d µs" % (1E6*(time.time()-start)))

707 µs


We can also save the whole object as a pickle which is useful for larger bacterial genomes which take minutes to instantiate

In [346]:
covid.save_pickle('covid.pkl')

PicklingError: Can't pickle <class 'gumpy.genome.Genome'>: it's not the same object as gumpy.genome.Genome

The protein-coding genes, loci and RNA-coding genes are extracted from the GenBank file and held in a dictionary called `features` 

In [351]:
covid.genes['S']

S gene
3829 nucleotides, codes for protein
acgaa...gaaca
-7 -6 -5 -4 -3 ...-5 -4 -3 -2 -1 
MFVFL...LHYT!
1 2 3 4 5 ...1270 1271 1272 1273 1274 

Since the Covid is ssRNA, none of these genes can be on the reverse strand (because there is no reverse strand) so they all have `'reverse_complement'` marked `False`. The situation would be very different in *M. tuberculosis* where several important genes, notably *katG*, can be found on the reverse strand.

Now this is the complicated bit; genes can overlap (like the example above at `index==1000`. Hence if we want to create a `numpy` array as long as the genome that tells us which features(s) each nucleotide belongs to, we cannot (necessarily) use a simple 1D array. Instead, as we go through each `feature`, we have to see if it can be accommodated in the existing array. If it overlaps with a feature we've already added to the array, then we have to add an extra row to the numpy array.

For the SARS-CoV-2 genome, 2 rows is sufficient to allow for all overlapping features (genes) to be uniquely labelled.

In [353]:
covid.stacked_gene_name

array([['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', '']], dtype='<U20')

There is a whole range of other similar `feature_foo` arrays that work in this way and are setup now to make identifying features (genes) easier later on. Note that cds = coding sequence i.e. does it encode amino acids (and thence a protein). If it does, then the sequence will form triplets (codons).

In [354]:
covid.stacked_is_cds

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [355]:
covid.stacked_is_reverse_complement

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [356]:
covid.stacked_is_promoter

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [357]:
covid.stacked_nucleotide_number

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [360]:
covid.stacked_nucleotide_sequence[covid.stacked_is_cds]

array(['a', 't', 'g', ..., 't', 'a', 'a'], dtype='<U1')

As an example, let's build the nucleotide sequence of the Spike protein (whose gene is called *S*).

Since in this case the `covid.feature_name` array has shape (2,29903) and `covid.sequence` has shape (29903,) we have to collapse the former so we can make a mask to fancy index the latter.

In [361]:
mask=numpy.any(covid.stacked_gene_name=='S',axis=0)
covid.nucleotide_sequence[mask]

array(['a', 'c', 'g', ..., 't', 'a', 'a'], dtype='<U1')

Then we could simply make a string of the sequence of the *S* gene

In [362]:
''.join(i for i in covid.nucleotide_sequence[mask])

'acgaacaatgtttgtttttcttgttttattgccactagtctctagtcagtgtgttaatcttacaaccagaactcaattaccccctgcatacactaattctttcacacgtggtgtttattaccctgacaaagttttcagatcctcagttttacattcaactcaggacttgttcttacctttcttttccaatgttacttggttccatgctatacatgtctctgggaccaatggtactaagaggtttgataaccctgtcctaccatttaatgatggtgtttattttgcttccactgagaagtctaacataataagaggctggatttttggtactactttagattcgaagacccagtccctacttattgttaataacgctactaatgttgttattaaagtctgtgaatttcaattttgtaatgatccatttttgggtgtttattaccacaaaaacaacaaaagttggatggaaagtgagttcagagtttattctagtgcgaataattgcacttttgaatatgtctctcagccttttcttatggaccttgaaggaaaacagggtaatttcaaaaatcttagggaatttgtgtttaagaatattgatggttattttaaaatatattctaagcacacgcctattaatttagtgcgtgatctccctcagggtttttcggctttagaaccattggtagatttgccaataggtattaacatcactaggtttcaaactttacttgctttacatagaagttatttgactcctggtgattcttcttcaggttggacagctggtgctgcagcttattatgtgggttatcttcaacctaggacttttctattaaaatataatgaaaatggaaccattacagatgctgtagactgtgcacttgaccctctctcagaaacaaagtgtacgttgaaatccttcactgtagaaaaaggaatctatcaaacttctaactttagagtccaaccaacagaatctattgttagatttcctaa

This is fine, but we really want to expose a gene/protein-centric view so it makes sense for the `Genome` object to hold a large number of `Gene` objects

In [364]:
sample=copy.deepcopy(covid)
sample.nucleotide_sequence[sample.nucleotide_index==4]='t'
sample.is_indel[sample.nucleotide_index==100]=True
sample.indel_length[sample.nucleotide_index==100]=3
sample.nucleotide_sequence[sample.nucleotide_index==22000]='g'
sample.is_indel[sample.nucleotide_index==22100]=True
sample.indel_length[sample.nucleotide_index==22100]=-5
sample._recreate_genes()

The main design principle for the `Genome` object is to overload `__sub__` so it returns a list of genome indices where this some kind of difference (e.g. 

In [366]:
sample-covid

array([    4,   100, 22000, 22100])

In [367]:
covid.genes['S']

S gene
3829 nucleotides, codes for protein
acgaa...gaaca
-7 -6 -5 -4 -3 ...-5 -4 -3 -2 -1 
MFVFL...LHYT!
1 2 3 4 5 ...1270 1271 1272 1273 1274 

In [368]:
sample.genes['S'].list_mutations_wrt(covid.genes['S'])

['H146Q', '538_indel']

In [369]:
sample.genes['S']-covid.genes['S']

array([438, 538])

In [ ]:
sample.genes['S'].amino_acid_sequence